In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!pip install ipython-autotime
%load_ext autotime

  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
time: 1.97 ms (started: 2024-08-18 14:54:31 +00:00)


In [3]:
!pip install essentia

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 26.3 MB/s eta 0:00:00
time: 12.4 s (started: 2024-08-18 14:54:31 +00:00)


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
time: 1min 30s (started: 2024-08-18 14:54:44 +00:00)


### 1. Extract Spectrograms from audio files and create dataset with computer audio and error condition labels

In [5]:
log_file_path = '/content/drive/MyDrive/THESIS/thesis_data/THESIS_DATA/spectrogram_images_dataset_p14/spec_processing_errors_img.log'
processed_files_log = '/content/drive/MyDrive/THESIS/thesis_data/THESIS_DATA/spectrogram_images_dataset_p14/processed_files_img.txt'


time: 1.02 ms (started: 2024-08-18 14:56:52 +00:00)


In [12]:
import os
import csv
import numpy as np
import matplotlib.pyplot as plt
import logging
import essentia.standard as es

def extract_spectrogram(file_path, frame_size=2048, hop_size=512):
    """Extracts the spectrogram from an audio file using Essentia."""
    try:
        loader = es.MonoLoader(filename=file_path)
        audio = loader()

        windowing = es.Windowing(type='hann')
        spectrum = es.Spectrum()

        spectrogram = []
        for frame in es.FrameGenerator(audio, frameSize=frame_size, hopSize=hop_size, startFromZero=True):
            windowed_frame = windowing(frame)
            spectrogram.append(spectrum(windowed_frame))

        return np.array(spectrogram).T
    except Exception as e:
        logging.error(f"Error processing file {file_path}: {e}")
        return None

def save_spectrogram_as_image(spectrogram, output_image_path):
    """Saves the spectrogram as an image."""
    plt.figure(figsize=(10, 4))
    plt.imshow(spectrogram, aspect='auto', origin='lower', cmap='viridis')
    plt.axis('off')  # Remove axes
    plt.savefig(output_image_path, bbox_inches='tight', pad_inches=0)
    plt.close()

def create_dataset_with_spectrograms(root_directory, csv_file_path, output_image_dir):
    """Creates a dataset of spectrograms from 'computer' audio files, labeled by 'error_condition',
    saves the spectrograms as images, and saves the paths and labels to a CSV file."""

    # Ensure the directory for storing images exists
    os.makedirs(output_image_dir, exist_ok=True)

    # Load the list of processed files
    processed_files = set()
    if os.path.exists(processed_files_log):
        with open(processed_files_log, 'r') as f:
            processed_files = set(line.strip() for line in f)

    # Open a CSV file for storing image paths and labels
    with open(csv_file_path, mode='a', newline='') as file:
        writer = csv.writer(file)
        if os.path.getsize(csv_file_path) == 0:
            writer.writerow(["image_path", "label"])  # Write header if the file is empty

        # Iterate through phonation mode folders
        for phonation_mode in os.listdir(root_directory):
            phonation_mode_path = os.path.join(root_directory, phonation_mode)

            if os.path.isdir(phonation_mode_path):
                # Iterate through error condition folders
                for error_condition in os.listdir(phonation_mode_path):
                    error_condition_path = os.path.join(phonation_mode_path, error_condition)

                    if os.path.isdir(error_condition_path):
                        for phrase_type in os.listdir(error_condition_path):
                            phrase_type_path = os.path.join(error_condition_path, phrase_type)

                            if os.path.isdir(phrase_type_path):
                                for clip_number in os.listdir(phrase_type_path):
                                    clip_number_path = os.path.join(phrase_type_path, clip_number)

                                    if os.path.isdir(clip_number_path):
                                        # Look for the 'computer' audio file within this clip folder
                                        for file_name in os.listdir(clip_number_path):
                                            if "computer" in file_name and file_name.endswith('.wav'):
                                                file_path = os.path.join(clip_number_path, file_name)

                                                image_filename = f"{phonation_mode}_{error_condition}_{phrase_type}_{clip_number}.png"
                                                image_filepath = os.path.join(output_image_dir, image_filename)

                                                # Check if the image file already exists
                                                if image_filename in processed_files:
                                                    continue

                                                try:
                                                    # Extract the spectrogram
                                                    spectrogram = extract_spectrogram(file_path)

                                                    if spectrogram is not None:
                                                        # Save the spectrogram as an image
                                                        save_spectrogram_as_image(spectrogram, image_filepath)

                                                        # Write the path and label to the CSV
                                                        writer.writerow([image_filepath, error_condition])

                                                        # Add the processed file to the set
                                                        processed_files.add(image_filename)
                                                except Exception as e:
                                                    logging.error(f"Error processing file {file_path}: {e}")
                                                    continue

    # Save the list of processed files
    with open(processed_files_log, 'w') as f:
        for filename in processed_files:
            f.write(f"{filename}\n")


time: 1.6 s (started: 2024-08-18 15:00:17 +00:00)


In [13]:
root_directory = "/content/drive/MyDrive/THESIS/thesis_data/THESIS_DATA/intermediate/p_14"
csv_file_path = "/content/drive/MyDrive/THESIS/thesis_data/THESIS_DATA/spectrogram_images_dataset_p14/spectrogram_images_dataset_p14.csv"
output_dir = "/content/drive/MyDrive/THESIS/thesis_data/THESIS_DATA/spectrogram_images_dataset_p14"
create_dataset_with_spectrograms(root_directory, csv_file_path, output_dir)

time: 11min 20s (started: 2024-08-18 15:00:24 +00:00)


## Build and Train Model

In [45]:
csv_file_path = "/content/drive/MyDrive/THESIS/thesis_data/THESIS_DATA/spectrogram_images_dataset_p14/spectrogram_images_dataset_p14.csv"

time: 1.99 ms (started: 2024-08-18 17:47:54 +00:00)


In [46]:
import os
import csv
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.optimizers import Adam

# Parameters
image_size = (128, 128)  # Resize all images to this size
test_size = 0.2

# Load CSV file
data = pd.read_csv(csv_file_path)

# Extract image paths and labels
image_paths = data['image_path'].values
labels = data['label'].values

# Encode labels
le = LabelEncoder()
encoded_labels = le.fit_transform(labels)
categorical_labels = to_categorical(encoded_labels)

# Load and preprocess images
images = []
for img_path in image_paths:
    img = load_img(img_path, target_size=image_size, color_mode='rgb')
    img_array = img_to_array(img) / 255.0  # Normalize to [0, 1]
    images.append(img_array)

images = np.array(images)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, categorical_labels, test_size=test_size, random_state=42)

time: 5.72 s (started: 2024-08-18 17:47:55 +00:00)


###Trying Keras CNN

In [67]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the model
model = Sequential()

# Convolutional layers
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the output and add dense layers
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(le.classes_), activation='softmax'))  # Output layer with softmax

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Summary of the model
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_22 (Conv2D)                   │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_21 (MaxPooling2D)      │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_23 (Conv2D)                   │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_22 (MaxPooling2D)      │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_24 (Conv2D)                   │ (None, 28, 28, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_23 (MaxPooling2D)      │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_7 (Flatten)                  │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_19 (Dense)                     │ (None, 128)                 │       3,211,392 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_20 (Dense)                     │ (None, 7)                   │             903 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,305,543 (12.61 MB)

 Trainable params: 3,305,543 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

time: 172 ms (started: 2024-08-18 19:58:57 +00:00)


In [68]:
from tensorflow.keras.callbacks import EarlyStopping

# Define early stopping
early_stopping = EarlyStopping(monitor='val_accuracy',  # Monitor validation loss
                               patience=10,          # Number of epochs with no improvement
                               restore_best_weights=True)

time: 1.49 ms (started: 2024-08-18 19:59:01 +00:00)


In [62]:
# Train the model with early stopping
history = model.fit(X_train, y_train,
                    epochs=200,
                    batch_size=32,
                    validation_split=0.2,
                    callbacks=[early_stopping])

Epoch 1/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step - accuracy: 0.1549 - loss: 1.9465 - val_accuracy: 0.1354 - val_loss: 1.9434
Epoch 2/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 0.1796 - loss: 1.9394 - val_accuracy: 0.1354 - val_loss: 1.9423
Epoch 3/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 11s 864ms/step - accuracy: 0.1738 - loss: 1.9423 - val_accuracy: 0.1354 - val_loss: 1.9420
Epoch 4/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 22s 1s/step - accuracy: 0.1588 - loss: 1.9486 - val_accuracy: 0.1562 - val_loss: 1.9427
Epoch 5/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 19s 910ms/step - accuracy: 0.1440 - loss: 1.9438 - val_accuracy: 0.1458 - val_loss: 1.9444
Epoch 6/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 12s 965ms/step - accuracy: 0.1194 - loss: 1.9470 - val_accuracy: 0.1354 - val_loss: 1.9430
Epoch 7/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 0.1758 - loss: 1.9465 - val_accuracy: 0.1354 - val_loss: 1.9427
Epoch 8/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 19s 920ms/step - accuracy: 0.1835 - loss: 1.9452 - val_accuracy

In [66]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_accuracy:.4f}")
print(f"Test loss: {test_loss:.4f}")

# Save the model
# model.save('spectrogram_classification_model.h5')

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 206ms/step - accuracy: 0.0958 - loss: 1.9540
Test accuracy: 0.0833
Test loss: 1.9537
time: 1.02 s (started: 2024-08-18 19:58:43 +00:00)


Without early stopping callback

In [69]:
# Train the model with early stopping
history = model.fit(X_train, y_train,
                    epochs=200,
                    batch_size=32,
                    validation_split=0.2)

Epoch 1/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 16s 1s/step - accuracy: 0.1690 - loss: 1.9513 - val_accuracy: 0.1354 - val_loss: 1.9412
Epoch 2/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - accuracy: 0.1567 - loss: 1.9453 - val_accuracy: 0.1354 - val_loss: 1.9446
Epoch 3/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - accuracy: 0.1746 - loss: 1.9453 - val_accuracy: 0.1354 - val_loss: 1.9454
Epoch 4/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - accuracy: 0.1590 - loss: 1.9443 - val_accuracy: 0.1354 - val_loss: 1.9469
Epoch 5/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.1021 - loss: 1.9493 - val_accuracy: 0.1354 - val_loss: 1.9433
Epoch 6/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - accuracy: 0.1827 - loss: 1.9435 - val_accuracy: 0.1354 - val_loss: 1.9428
Epoch 7/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 15s 923ms/step - accuracy: 0.1374 - loss: 1.9439 - val_accuracy: 0.1354 - val_loss: 1.9439
Epoch 8/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 22s 1s/step - accuracy: 0.1491 - loss: 1.9462 - val_accuracy: 0.1354 

In [71]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_accuracy:.4f}")
print(f"Test loss: {test_loss:.4f}")

4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 367ms/step - accuracy: 0.3015 - loss: 2.5525
Test accuracy: 0.3083
Test loss: 2.4654
time: 2.84 s (started: 2024-08-18 20:58:06 +00:00)


## ResNet

In [33]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Parameters
image_size = (224, 224)  # Resize all images to this size for ResNet
num_frames = 10  # Number of frames to use for each sequence

# Load CSV file
data = pd.read_csv(csv_file_path)

# Extract image paths and labels
image_paths = data['image_path'].values
labels = data['label'].values

# Encode labels
le = LabelEncoder()
encoded_labels = le.fit_transform(labels)
categorical_labels = to_categorical(encoded_labels)

def load_and_preprocess_image(image_path, image_size):
    """Load and preprocess the image."""
    img = load_img(image_path, target_size=image_size)
    img_array = img_to_array(img) / 255.0  # Normalize to [0, 1]
    return img_array

# Load and preprocess images
X = np.array([load_and_preprocess_image(path, image_size) for path in image_paths])
y = np.array(encoded_labels)  # Use encoded labels directly here

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


time: 10.4 s (started: 2024-08-18 16:07:45 +00:00)


In [34]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

# Load the pre-trained ResNet50 model without the top layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Build the model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dense(len(le.classes_), activation='softmax')
])

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
time: 3.84 s (started: 2024-08-18 16:08:30 +00:00)


In [36]:
# Compile the model
from tensorflow.keras.optimizers import Adam

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


# Train the model
history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_split=0.2, callbacks=[early_stopping])


Epoch 1/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 101s 7s/step - accuracy: 0.1477 - loss: 2.0289 - val_accuracy: 0.1667 - val_loss: 1.9457
Epoch 2/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 83s 7s/step - accuracy: 0.1562 - loss: 1.9493 - val_accuracy: 0.1667 - val_loss: 1.9479
Epoch 3/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 173s 10s/step - accuracy: 0.1992 - loss: 1.9424 - val_accuracy: 0.1354 - val_loss: 1.9485
Epoch 4/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 114s 7s/step - accuracy: 0.1493 - loss: 1.9445 - val_accuracy: 0.1354 - val_loss: 1.9530
Epoch 5/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 137s 7s/step - accuracy: 0.1654 - loss: 1.9469 - val_accuracy: 0.1354 - val_loss: 1.9433
Epoch 6/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 85s 7s/step - accuracy: 0.1436 - loss: 1.9452 - val_accuracy: 0.2083 - val_loss: 1.9369
Epoch 7/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 142s 7s/step - accuracy: 0.1535 - loss: 1.9465 - val_accuracy: 0.1354 - val_loss: 1.9456
Epoch 8/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 140s 7s/step - accuracy: 0.1729 - loss: 1.9450 - val_accuracy: 0.1354 - va

In [37]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_accuracy:.2f}")


4/4 ━━━━━━━━━━━━━━━━━━━━ 22s 5s/step - accuracy: 0.1377 - loss: 1.9619
Test accuracy: 0.13
time: 22.9 s (started: 2024-08-18 16:30:12 +00:00)
